<a href="https://colab.research.google.com/github/shreyabhadwal/AI_Trash_System/blob/main/ObjectDet_Trash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import the required libraries

In [6]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

In [18]:
!pip install folium==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!sudo apt-get install libportaudio2
!pip install sounddevice

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (151 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


In [10]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Mount the drive and unzip the dataset

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!unzip '/content/drive/MyDrive/Dataset.zip'

Archive:  /content/drive/MyDrive/Dataset.zip
   creating: Dataset/Test/
  inflating: Dataset/Test/glass121.jpg  
  inflating: Dataset/Test/glass121.xml  
  inflating: Dataset/Test/glass122.jpg  
  inflating: Dataset/Test/glass122.xml  
  inflating: Dataset/Test/glass123.jpg  
  inflating: Dataset/Test/glass123.xml  
  inflating: Dataset/Test/glass124.jpg  
  inflating: Dataset/Test/glass124.xml  
  inflating: Dataset/Test/glass125.jpg  
  inflating: Dataset/Test/glass125.xml  
  inflating: Dataset/Test/glass126.jpg  
  inflating: Dataset/Test/glass126.xml  
  inflating: Dataset/Test/glass127.jpg  
  inflating: Dataset/Test/glass127.xml  
  inflating: Dataset/Test/glass128.jpg  
  inflating: Dataset/Test/glass128.xml  
  inflating: Dataset/Test/glass129.jpg  
  inflating: Dataset/Test/glass129.xml  
  inflating: Dataset/Test/glass130.jpg  
  inflating: Dataset/Test/glass130.xml  
  inflating: Dataset/Test/glass131.jpg  
  inflating: Dataset/Test/glass131.xml  
  inflating: Dataset/Test/

### Load the Dataset

In [13]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'Dataset/Train',
    'Dataset/Train',
    ['Glass R', 'Metal R','Plastic R','Paper R']
)

In [14]:
val_data = object_detector.DataLoader.from_pascal_voc(
    'Dataset/Test',
    'Dataset/Test',
    ['Glass R', 'Metal R','Plastic R','Paper R']
)

### Select the model architecture

Here, we have used the EfficientDet_Lite0 architecture. 

In [15]:
spec = model_spec.get('efficientdet_lite0')

### Train the model

In [16]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=30, validation_data=val_data)

Epoch 1/30
120/120 [==============================] - 291s 2s/step - det_loss: 1.2826 - cls_loss: 0.9152 - box_loss: 0.0073 - reg_l2_loss: 0.0634 - loss: 1.3461 - learning_rate: 0.0065 - gradient_norm: 2.9535 - val_det_loss: 0.8696 - val_cls_loss: 0.5916 - val_box_loss: 0.0056 - val_reg_l2_loss: 0.0635 - val_loss: 0.9331
Epoch 2/30
120/120 [==============================] - 253s 2s/step - det_loss: 0.7751 - cls_loss: 0.5947 - box_loss: 0.0036 - reg_l2_loss: 0.0635 - loss: 0.8386 - learning_rate: 0.0050 - gradient_norm: 3.1737 - val_det_loss: 0.6691 - val_cls_loss: 0.4626 - val_box_loss: 0.0041 - val_reg_l2_loss: 0.0636 - val_loss: 0.7327
Epoch 3/30
120/120 [==============================] - 255s 2s/step - det_loss: 0.6469 - cls_loss: 0.5098 - box_loss: 0.0027 - reg_l2_loss: 0.0636 - loss: 0.7105 - learning_rate: 0.0049 - gradient_norm: 3.1968 - val_det_loss: 0.5785 - val_cls_loss: 0.4230 - val_box_loss: 0.0031 - val_reg_l2_loss: 0.0636 - val_loss: 0.6421
Epoch 4/30
120/120 [===========

### Evaluating the model

In [17]:
model.evaluate(val_data)

2/2 [==============================] - 24s 5s/step



{'AP': 0.6790275,
 'AP50': 0.9025529,
 'AP75': 0.7973942,
 'AP_/Glass R': 0.5836174,
 'AP_/Metal R': 0.72330254,
 'AP_/Paper R': 0.7909178,
 'AP_/Plastic R': 0.6182721,
 'APl': 0.67925054,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.81640625,
 'ARm': -1.0,
 'ARmax1': 0.78125,
 'ARmax10': 0.81640625,
 'ARmax100': 0.81640625,
 'ARs': -1.0}

### Exporting the model as TensorFlow Lite Model to be run on Raspberry Pi

In [ ]:
model.export(export_dir='.', tflite_filename='android.tflite')